## In this notebook we read in timestamped data of stars to repos. We select some top repos by total stars, then plot the timeseries of stars they received every month since December 2014.

Here we read in the data directly from a curated CSV. It is also possible to query this data from the database directly. 


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt


In [ ]:
df=pd.read_csv('data/usage.csv')
df['timestamp']=pd.to_datetime(df['timestamp'])

In [ ]:
df.head()

In [ ]:
#selecting the J'th through K'th top repos by their all-time highest star count
j= 3
k = 5
topk=df.groupby('repo_id')['stars'].max().sort_values().tail(k).head(j)
print(topk)

In [ ]:
topk_repoids=list(topk.index)

In [ ]:
for repo in topk_repoids:
    repo_df = df[df['repo_id']==repo].set_index('timestamp')
    repo_df=repo_df.loc[(repo_df.index > '2014-01-01 00:00:00')&(repo_df.index < '2022-09-01 00:00:00')]
    f,ax=plt.subplots(1,2,figsize=(10,4))
    repo_df.stars.plot(color='red',ax=ax[0])
    ax[0].set_ylabel('Stars',size=13)
    ax[0].semilogy()
    ax[0].set_xlabel('')
    ax[0].set_ylim(bottom=10)
    repo_df.downloads.plot(color='blue',ax=ax[1])
    ax[1].set_ylabel('Downloads',size=13)
    ax[1].semilogy()
    ax[1].set_xlabel('')
    ax[1].set_ylim(bottom=10)
    plt.suptitle('Repo ID: '+str(repo),size=13)
    ax[0].grid()
    ax[1].grid()
    plt.show()
    print('---------')

## The monthly dynamics of stars are rather volatile. In some months a repo can get many times more stars than in another month.

In [ ]:
plt.figure(figsize=(10,8))
for repo in topk_repoids:
    repo_df = df[df['repo_id']==repo].set_index('timestamp')
    repo_df=repo_df.loc[(repo_df.index > '2014-01-01 00:00:00')&(repo_df.index < '2022-09-01 00:00:00')]
    repo_df.stars.diff().plot(label=repo)
    
plt.ylabel('Stars',size=14)
plt.xlabel('')
plt.grid()
plt.semilogy()
plt.ylim(bottom=1)

plt.legend(prop={'size':14})

## We can recreate this plot for Downloads - a measure of use rather than popular success or visibility.

In [ ]:
plt.figure(figsize=(6,5))
linestyles=['--',':','-']
for repo in topk_repoids:
    repo_df = df[df['repo_id']==repo].set_index('timestamp')
    repo_df=repo_df.loc[(repo_df.index > '2014-01-01 00:00:00')&(repo_df.index < '2022-09-01 00:00:00')]
    repo_df.downloads.diff().plot(label=repo,linestyle=linestyles.pop(),color='black')
    
plt.ylabel('Downloads',size=14)
plt.xlabel('')
plt.semilogy()
plt.yticks(size=14)
plt.xticks(size=14)

plt.grid()
plt.legend(title="Repo ID",prop={'size':14},loc='upper left')
plt.tight_layout()
plt.savefig('plots/DL_ts.pdf')

### Finally, we present a time series of new pull requests, quantifying collaborative code contributions, for two popular libraries.

In [ ]:
plt.figure(figsize=(6,5))
linestyles=['--',':','-']


repo_df = df[df['repo_id']==30514].set_index('timestamp')
repo_df=repo_df.loc[(repo_df.index > '2014-01-01 00:00:00')&(repo_df.index < '2022-09-01 00:00:00')]
repo_df.pullrequests.diff().plot(label='30514',
                                  linestyle=linestyles.pop(),color='black')


repo_df = df[df['repo_id']==27416].set_index('timestamp')
repo_df=repo_df.loc[(repo_df.index > '2014-01-01 00:00:00')&(repo_df.index < '2022-09-01 00:00:00')]
repo_df.pullrequests.diff().plot(label='27416',
                                  linestyle=linestyles.pop(),color='black')



plt.ylabel('New Pull Requests',size=14)
plt.xlabel('')
plt.yticks(size=14)
plt.xticks(size=14)


plt.grid()
plt.legend(title="Repo",prop={'size':14},loc='upper left')
plt.tight_layout()
plt.savefig('plots/pr_ts.pdf')